In [1]:
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
from diffusers import DiffusionPipeline, StableDiffusionPipeline
import transformers

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import RandAugment
from IPython.core.debugger import set_trace
import os

import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model_id = "CompVis/stable-diffusion-v1-4"

In [5]:
fixed_model = StableDiffusionPipeline.from_pretrained(model_id,
                                                torch_dtype=torch.float16,
                                                use_safetensors=True,
                                                variant="fp16")
fixed_model.to(device)

fine_tuned_model = DiffusionPipeline.from_pretrained(model_id,
                                                torch_dtype=torch.float16,
                                                use_safetensors=True,
                                                variant="fp16")
fine_tuned_model.to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.27.2",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [7]:
prior_images = []
prompt = "A dog"
for i in range(5):
    prior_images.append(fixed_model(prompt=prompt).images[0])

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
folder_path = '/home/ahm247/dreambooth/dataset/dog6'
input_images = []
for filename in os.listdir(folder_path): 
    img = Image.open(os.path.join(folder_path, filename))
    input_images.append(img)

In [21]:
fine_tuned_images_id = []
fine_tuned_images_pr = []
fine_tuned_prompt = 'A sks dog' #Note that sks is our rare token
for i in range(5): 
    fine_tuned_images_id.append(fine_tuned_model(prompt=fine_tuned_prompt).images[0])
    fine_tuned_images_pr.append(fine_tuned_model(prompt=prompt).images[0])

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
for i in range(len(fine_tuned_images_id)): 
    fine_tuned_images_id[i].save('image' + str(i) + '.jpg')